# Post-Processing Model Tuning Experiments

This Jupyter notebook contains hands-on learning activities for the DeapSECURE's "[Deep Learning (Neural Networks)](https://deapsecure.gitlab.io/deapsecure-lesson04-nn/)" module, Episode 7: "[Effective Deep Learning Workflow on HPC](https://deapsecure.gitlab.io/deapsecure-lesson04-nn/31-batch-tuning-hpc/index.html)". Please visit the [DeapSECURE website](https://deapsecure.gitlab.io/) to learn more about our training program.

## Introduction

After we ran all the model tuning experiments on HPC, now it is time to start recapping the results, and later analyze the findings.
Each hyperparameter set in these experiments correspond to a single model training run, and the outputs are stored in a separate folder that has been systematically named.

The final objective of the postprocessing stage is to gather all the results from these individual model trainings to create intermediate tables that will be further analyzed in the next stage ("post-analysis").

However, prior to doing this, we must validate the model training runs that were submitted in batch mode.
At the end of these runs, the training histories were stored in `model_history.csv` files.
We must inspect at the progress of the training did not show anything anomalous, such as lack of convergence, overfitting, etc.

## Import modules and define helper functions

In [ ]:
import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
from sherlock_ML_toolbox import fn_out_history_1H, fn_out_history_XH

The `plot_training_history` function creates a two-panel plot to visually inspect the training history in terms of the loss (left panel) and accuracy (right panel).
This will aid identification of abnormal training outcomes.

In [ ]:
def plot_training_history(history_file, subtitle):
    """
    Creates and displays a two-panel subplots to visualize the progress
    of a model training run.
    The left panel contains the training and validation loss vs. epochs;
    the right panel contains the training validation accuracy vs. epochs.
    This function expects to read the training history from a CSV file,
    which contains loss & accuracy values computed with training and validation
    sets.
    
    Args:
      history_file (str): The pathname of the CSV file.
      subtitle (str): A string to append to the plot's titles.
    """
    # Initialize the subplots
    fig, axs = plt.subplots(1, 2, figsize=(10,4))
    
    # The history file contains the data to plot
    epochMetrics = pd.read_csv(history_file)
    # The epoch values corresponds to the index of the read dataframe,
    # which will be 0, 1, 2, ...
    epochs = np.array(epochMetrics.index)

    # Plot the loss subplot
    axs[0].plot(epochs, epochMetrics['loss'])
    axs[0].plot(epochs+1, epochMetrics['val_loss'])

    # Code to add the title, axis labels, etc.
    axs[0].set_title("Model Loss: " + subtitle)
    axs[0].legend(["Train Loss", "Val Loss"])
    axs[0].set_ylabel("Loss")
    axs[0].set_xlabel("Epochs")
    axs[0].set_xlim(xmin=0)
    axs[0].set_ylim(ymin=0)

    # Plot the accuracy subplot
    axs[1].plot(epochs, epochMetrics['accuracy'])
    axs[1].plot(epochs+1, epochMetrics['val_accuracy'])

    # Code to add the title, axis labels, etc.
    axs[1].set_title("Model Accuracy: " + subtitle)
    axs[1].legend(["Train Accuracy", "Val Accuracy"])
    axs[1].set_ylabel("Accuracy")
    axs[1].set_xlabel("Epochs")
    axs[1].set_xlim(xmin=0)

    # create space between the plots
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, \
                        wspace=0.4, hspace=0.4) 
    plt.show()

## Hidden Neurons Experiment

In this experiment, we trained many classifier models which have varied values of `hidden_neurons`, the number of neurons in the (only) hidden layer of the model.
Our goal is to observe the effect of `hidden_neurons` hyperparameter on the accuracy of the model.
The model training results are stored in subfolders (one folder per model training) under the `scan-hidden-neurons` folder.

Once the experimental results are collected, this postprocessing stage gathers all these results, validates them, and creates the intermediate data for final analysis.
This stage consists of the following steps:

1. First, scan all the results within the `scan-hidden-neurons` folder.

2. Validate the model-training runs, to ensure that they show normal behavior.
   (For example, we want to watch out for non-convergence, overfitting, or other anomalies.)

3. Extract the final metrics (loss & accuracy) from the trained models from the last epoch, create a dataframe to combine these metrics from multiple runs, and save this into an intermediate CSV table.

In the post-analysis stage (the next one) we will obtain the insight on how these metrics change as a result of the hyperparameter variations.
From this stage, we will determine the optimal set of hyperparameters for the `sherlock_18apps` classifier.

### Step 1: Discover and Load the Results

Let's inspect the content of one history file and determine the steps needed to get the values that we need:

In [ ]:
# Inspect what the last epochMetrics looks like (from the "baseline model run")
epochMetrics = pd.read_csv("scan-hidden-neurons/model_1H18N_lr0.0003_bs32_e30/model_history.csv")

**EXERCISE**: Examine the `epochMetrics` data structure and the contents (at least the "head" and the "tail").

In [ ]:
#TODO
#epochMetrix.info()

In [ ]:
#epochMetrix.#TODO

In [ ]:
#INFO
epochMetrics.head()

In [ ]:
#INFO
epochMetrics.tail()

The `plot_training_history` function will be used to visualize the progress of the training (the loss and accuracy):

In [ ]:
plot_training_history("scan-hidden-neurons/model_1H18N_lr0.0003_bs32_e30/model_history.csv",
                      subtitle="1H18N baseline")

**EXERCISE**: These plots show the behavior of the loss and accuracy in a normal training run.
Please study the behavior of these metrics above as a function of the number of epochs, and write a few descriptive sentences regarding what you see.

> **ANSWER**: #TODO

**QUESTION**: How to fetch the last row of this dataframe (labeled **29** above)?

> **ANSWER**: #TODO

In [ ]:
"""Fetch the last row of this dataframe, which is row labeled **29**""";

#epochMetrics.#TODO

Try convert the row data into a dict, because we will add one more column later:

In [ ]:
epochMetrics.iloc[29,:].to_dict()

#### Systematic File Naming

Because the filenames are designed to be systematic, it is best to encode the filename construction as a function.
The function arguments would be the hyperparameters (num of hidden neurons, learning rate, batch size), as well as num of epochs. In addition, a subdirectory prefix is provide because we store our runs in separate directory tree per experiment.
The `sherlock_ML_toolbox` module has a function named `fn_out_history_1H` to get the history file for a particular run:

In [ ]:
# Test the history filename function
fn_out_history_1H('scan-hidden-neurons', 18, 0.003, 32, 30) 

#### Result Discovery

Given the folder name pattern above, we can discover which runs have been accomplished in our own model tuning folders.
We can use `ls` and shell wildcard to find this out (the `-d` flag prevents `ls` from listing the contents of the directories):

In [ ]:
! ls -l -d scan-hidden-neurons/model_1H*N_lr*_bs*_e*/

Based on our run, we varied the value of `hidden_neurons` in this experiment, which can be found in the batch job submission script.
(Remember the contents of the `submit-scan-hidden-neurons.sh` script?)

In [ ]:
! cat scan-hidden-neurons/submit-scan-hidden-neurons.sh

(You may have more values of `hidden_neurons` that you tried; please look at your own runs.)

**EXERCISE** - Now create a list named `listHN` which contains the values of `hidden_neurons` used in your training runs:

In [ ]:
"""Enter the numbers of hidden neurons that you tried in your experiment,
see the listing of the `submit-scan-hidden-neurons.sh` script above.""";

#listHN = [ #TODO ]

In [ ]:
# base folder of the experiment
dirPathHN = "scan-hidden-neurons"

### Step 2: Validation of Model Training: Visual Inspection

The validation step is easiest done by graphically inspecting the training histories using the `plot_training_history` function.
We can do this for all the runs with the different numbers of hidden neurons by invoking a loop:

In [ ]:
for i, HN in enumerate(listHN):
    plot_training_history(fn_out_history_1H(dirPathHN, HN, 0.0003, 32, 30),
                          subtitle="1H"+str(HN)+"N")

**QUESTIONS**: Based on the plots shown above, inspect whether the training runs went as expected.

1) Visually inspect for any anomalies. In the answer box below, mark the runs that produce "abonrmal training trends", i.e. where the "loss vs epochs" and/or "accuracy vs epochs" curves exhibit a different behavior from what shown in the earlier 2-panel plot.

2) Visually (or numerically) check for convergence (e.g. check the loss or accuracy for the last 4-5 epochs; what their slopes look like in this region; any fluctuations?)

3) Observe the differences in the *final* accuracies as a result of different `hidden_neurons` values. (We will do this more carefully in the next phase)

> #### **ANSWERS**
>
> 1. Runs with `hidden_neurons` = #TODO... have odd-looking trend. Their curves look like #TODO ... (describe what you see)
>
> 2. #TODO
>
> 3. #TODO

This is the step where one can easily look at the graphics and determine which hyperparameter regime yield good results, and which regime should be avoided because they produce bad results.

### Step 3: Create a Result DataFrame for `hidden_neurons` Hyperparameter Scan

After validation and everything else,
what we want to analyze will be the changes of the final metrics
as the input hyperparameter is varied (in this case, `hidden_neurons`).
We will do this in the next notebook,
but for now, we will collect these final metrics
with the relevant hyperparameter(s) and metadata
into an intermediate dataframe and save them as a CSV file.

In the batch runs, we ran every model training with 30 epochs.
Based on the the validation above,
it is sufficient for now to collect the metrics at epoch=30
as the final metrics for each `hidden_neurons` hyperparameter.

In [ ]:
# This is the epoch number from which we want to extract results
# for analysis of hyperparameter effects
# (corresponding to epoch #30)
lastEpochNum = 29

Now we scan the results from the training runs in 

We use a loop to read all the output files from this experiment
and gather the final metrics into a new dataframe called `df_HN`
(where `HN` is, again, a shorthand for "hidden neurons tuning experiment").
The sequence of the varied hyperparameter specified in `listHN`,
defined aboves.

#### *Method 1: Via a temporary data structure*
 
In this method, we will construct and fill a temporary data structure (`all_lastEpochMetrics`) dynamically before forming the dataframe.
This approach is useful when the size of the data (e.g. total number of rows) is not known *a priori*.

The following is a *simplified* loop which shows the logic
of this intermediate data construction:

In [ ]:
all_lastEpochMetrics = []
# Fill in the rows for the DataFrame
for HN in listHN:
    # Read the history CSV file and get the last row's data,
    # which corresponds to the last epoch data.
    #run_subdir = "model_1H" + str(HN) + "N_lr0.0003_bs32_e30"
    #result_csv = os.path.join(dirPathHN, run_subdir, "model_history.csv")
    result_csv = fn_out_history_1H(dirPathHN, HN, 0.0003, 32, 30)
    print("Reading:", result_csv)
    epochMetrics = pd.read_csv(result_csv)
    # Fetch the loss, accuracy, val_loss, and val_accuracy from the last epoch
    # (should be the last row in the CSV file unless there's something wrong
    # during the traning)
    lastEpochMetrics = epochMetrics.iloc[lastEpochNum, :].to_dict()
    # Attach the "neurons" value
    lastEpochMetrics["hidden_neurons"] = HN
    all_lastEpochMetrics.append(lastEpochMetrics)

> In real-world cases, model training may employ "early stopping"
> criteria which may lead to different numbers of epochs
> for different training runs.
> In this case, you will have to modify the output reading algorithm
> to find the correct final epoch for each training run.

**EXERCISE**:
Look over the contents of `all_lastEpochMetrics` and verify if the data has been loaded correctly.

In [ ]:
# Compare against one result

! tail -n 1 scan-hidden-neurons/model_1H1024N_lr0.0003_bs32_e30/model_history.csv

Now construct the `df_HN` dataframe:

In [ ]:
df_HN = pd.DataFrame(all_lastEpochMetrics, 
                     columns=["hidden_neurons", "loss", "accuracy", "val_loss", "val_accuracy"])

In [ ]:
print(df_HN)

In [ ]:
# Just check the data structure is according to what we expect
df_HN.info()

### Step 4: Save the Intermediate DataFrame

In [ ]:
df_HN.to_csv("post_processing_hpc_neurons.csv", index=False)

This intermediate dataframe is good for further analysis.
We will analyze the data on a subsequent notebook.

Now let us postprocess the other experiments.

### *OPTIONAL: Generalized metrics loader:* `load_bulk_final_metrics`

The method above is fine to use *ad hoc*;
however, the resulting intermediate data lacks additional information.
Only `hidden_neurons` is stored with the dataframe,
whereas other hyperparameters (e.g. learning rate, ...) are not kept.
In batch computation, we want to keep track of which calculations (or experiments) produces which results.
All of these information bits need to be somehow preserved
when constructing an intermediate dataset for further analysis.
Otherwise, the contexts that are lost may not be recoverable later on;
consequently we could not trace back the origin of our results.
In scientific research, the lost context will compromise the *reproducibility*
of the experiment.

In this lesson, we choose to store these contexts together with the dataframe.
We will to store the following bits of information:

* All the hyperparameters used (whether varied or fixed in this experiment)
* "Experiment code", a short descriptive string that can quickly identify the type of experiment
* Batch job ID.

In this simplified analysis, we will omit batch job ID (`Job_ID`) below but prepare a column in the dataframe for your own exercise to fill.
This field is meant to store the job ID number assigned by SLURM, so that we can associate the result to the specific computation and outputs.
This is important to troubleshoot issues/errors
when our experiments grow big
(e.g. we scan very many hyperparameters).

In [ ]:
def load_bulk_final_metrics(expt_name, hyperparams_list, expt_code):
    """Load final metrics (final training outcomes) in bulk.
    For this lesson, "training outcomes" are defined as the metrics computed
    in the last epoch of the training.
    
    Args:
      expt_name (str): Base directory, whose name also should describe the experiment.
      hyperparams_list (list): A list of hyperparameter sets.
        Each hyperparameter set is defined as a 4-tuple of
        (hidden_neurons, learning_rate, batch_size, epochs)
        used to train a model.
      expt_code (str): A short string suffix to describe the experiment
        in the dataframe.
    """
    all_lastEpochMetrics = []
    Expt_IDs = []
    # Fill in the rows for the DataFrame
    for (HN, LR, BS, EPOCH) in hyperparams_list:
        # Read the history CSV file and get the last row's data,
        # which corresponds to the last epoch data.
        result_csv = fn_out_history_1H(expt_name, HN, LR, BS, EPOCH)
        print("Reading:", result_csv, flush=True)

        # Define which epoch we want to extract the metrics from:
        lastEpochNum = EPOCH-1
        epochMetrics = pd.read_csv(result_csv)
        # Fetch the loss, accuracy, val_loss, and val_accuracy from the last epoch
        # (should be the last row in the CSV file unless there's something wrong
        # during the traning)
        lastEpochMetrics = epochMetrics.iloc[lastEpochNum, :].to_dict()
        # Attach the "metadata" values (Model_Type, Job_ID, hyperparameters)
        lastEpochMetrics["hidden_neurons"] = HN
        lastEpochMetrics["learning_rate"] = LR
        lastEpochMetrics["batch_size"] = BS
        lastEpochMetrics["epoch"] = lastEpochNum
        lastEpochMetrics["Expt_ID"] = None  # Will fill this later
        lastEpochMetrics["Job_ID"] = None  # FIXME for advanced learners
        all_lastEpochMetrics.append(lastEpochMetrics)
        # Ad-hoc: Expt_IDs is a string column and has to be added separately below!
        # For some reason, initializing it with the rest of the columns
        # doesn't work because of a non-numerical datatype
        Expt_IDs.append(f"1H{HN}N{expt_code}")

    #print(all_lastEpochMetrics[:3])  # only for debugging
    # Construct the dataframe
    cols_outcomes = [
        "Expt_ID", "Job_ID", 
        "hidden_neurons", "learning_rate", "batch_size", "epoch",
        "loss", "accuracy", "val_loss", "val_accuracy"
    ]
    df_outcomes = pd.DataFrame(all_lastEpochMetrics, columns=cols_outcomes)
    # Attach the Model_Type here; it will automatically switch the dtype to the correct one
    # to support string:
    df_outcomes["Expt_ID"] = Expt_IDs
    return df_outcomes

The generalized final-metrics loader above relies on a list of hyperparameter sets to decide which run results to load.
Here is the way to

Now create an "extended" `df_HN` which has the complete hyperparameters & other metadata:

In [ ]:
df_HN_ext = load_bulk_final_metrics("scan-hidden-neurons",
                                    hyperparams_HN,
                                    "-neurons")

In [ ]:
df_HN_ext

In [ ]:
df_HN_ext.to_csv("post_processing_hpc_neurons_ext.csv", index=False)

## Learning Rate Experiment

Now that we have laid a good foundation with the "scan-hidden-neuron" experiment,
we will reapply the postprocessing to the "learning rate" experiment.

### Step 1: Discover and Load the Results (LR)

In [ ]:
! ls -l -d scan-learning-rate/model_1H*N_lr*_bs*_e*/

 **EXERCISE** - Now create a list named `listLR` which contains the values of `learning_rate`'s used in your training runs:

In [ ]:
"""Enter the learning rates that you tried in your experiment,
see the contents of the `submit-scan-learning-rate.sh` script in your working directory.""";

#dirPathLR = #TODO
#listLR = [ #TODO ]

### Step 2: Validation of Model Training: Visual Inspection (LR)

Let's repeat the visual validation of model training runs for the scan over learning rates:

In [ ]:
"""Repeat the validation of model training, but now for the learning rate experiment""";

#for i, LR in enumerate(#TODO):
#    plot_training_history(fn_out_history_1H(#TODO),
#                          subtitle="lr"+str(LR))

Again, for the set of results shown above, we will ask a similar set of questions.

**QUESTIONS**: Based on the plots shown above, inspect whether the training runs went as expected.

1) Visually inspect for any anomalies. In the answer box below, mark the runs that produce "abnormal training trends", i.e. where the "loss vs epochs" and/or "accuracy vs epochs" curves exhibit a different behavior from what shown in the earlier 2-panel plot.

2) Visually (or numerically) check for convergence (e.g. check the loss or accuracy for the last 4-5 epochs; what their slopes look like in this region; any fluctuations?)

3) Observe the differences in the *final* accuracies as a result of different `learning_rate` values. What happens with the metrics as we use bigger and bigger learning rate? (We will do this more carefully in the next phase.)

> #### **ANSWERS**
>
> 1. #TODO
>
> 2. #TODO
>
> 3. #TODO

### Step 3: Create an Intermediate DataFrame for `learning_rate` Scan

In [ ]:
"""Use the same loop structure we used earlier to parse the history data,
and create the temporary data structure which will be converted to
a dataframe called `df_LR` below.""";

#all_lastEpochMetrics = []
## Fill in the rows for the DataFrame
#for LR in listLR:
#    # Read the history CSV file and get the last row's data,
#    # which corresponds to the last epoch data.
#    result_csv = #TODO
#    print("Reading:", result_csv)
#    epochMetrics = pd.read_csv(result_csv)
#    # Fetch the loss, accuracy, val_loss, and val_accuracy from the last epoch
#    # (should be the last row in the CSV file unless there's something wrong
#    # during the traning)
#    lastEpochMetrics = epochMetrics.iloc[lastEpochNum, :].to_dict()
#    # Attach the "learning_rate" value
#    lastEpochMetrics["learning_rate"] = #TODO
#    all_lastEpochMetrics.append(lastEpochMetrics)

In [ ]:
"""Now construct df_LR:""";

#df_LR = pd.DataFrame(all_lastEpochMetrics, #TODO)

#df_LR

### Step 4: Save the Intermediate DataFrame (LR)

In [ ]:
df_LR.to_csv("post_processing_hpc_lr.csv", index=False)

## Batch Size Experiment

Follow the same steps as before, please replicate the recipe to check the effect of `batch_size` hyperparameter.


Now that we have laid a good foundation with the "scan-hidden-neuron" experiment,
we will reapply the postprocessing to the "learning rate" experiment.

### Step 1: Discover and Load the Results (BS)

In [ ]:
! ls -l -d scan-batch-size/model_1H*N_lr*_bs*_e*/

In [ ]:
"""Enter the batch sizes that you tried in your experiment,
see the contents of the `submit-scan-batch-size.sh` script in your working directory.""";

#dirPathBS = #TODO
#listBS = [ #TODO ]

### Step 2: Validation of Model Training: Visual Inspection (BS)

Let's repeat the visual validation of model training runs for the scan over batch sizes:

In [ ]:
"""The validation of model training for the batch size experiment""";

#for i, BS in enumerate(#TODO):
#    plot_training_history(fn_out_history_1H(#TODO),
#                          subtitle="bs"+str(BS))

**QUESTIONS**: Based on the plots shown above, inspect whether the training runs went as expected.

1) Visually inspect for any anomalies. In the answer box below, mark the runs that produce "abnormal training trends", i.e. where the "loss vs epochs" and/or "accuracy vs epochs" curves exhibit a different behavior from what shown in the first 2-panel plot in this notebook.

2) Visually (or numerically) check for convergence (e.g. check the loss or accuracy for the last 4-5 epochs; what their slopes look like in this region; any fluctuations?)

3) Observe the differences in the *final* accuracies as a result of different `batch_size` values. What happens with the metrics as we use bigger and bigger batch size? (We will do this more carefully in the next phase.)

> #### **ANSWERS**
>
> 1. #TODO
>
> 2. #TODO
>
> 3. #TODO

### Step 3: Create an Intermediate DataFrame for `batch_size` Scan

In [ ]:
"""Use the same loop structure we used earlier to parse the history data,
and create the temporary data structure which will be converted to
a dataframe called `df_BS` below.""";

#all_lastEpochMetrics = []
## Fill in the rows for the DataFrame
#for #TODO:
#    # Read the history CSV file and get the last row's data,
#    # which corresponds to the last epoch data.
#    #TODO
#    # Fetch the loss, accuracy, val_loss, and val_accuracy from the last epoch
#    # (should be the last row in the CSV file unless there's something wrong
#    # during the traning)
#    #TODO
#    # Attach the "batch_size" value
#    #TODO
#    all_lastEpochMetrics.append(lastEpochMetrics)

In [ ]:
all_lastEpochMetrics = []
# Fill in the rows for the DataFrame
for BS in listBS:
    # Read the history CSV file and get the last row's data,
    # which corresponds to the last epoch data.
    result_csv = fn_out_history_1H(dirPathBS, 18, 0.0003, BS, 30)
    print("Reading:", result_csv)
    epochMetrics = pd.read_csv(result_csv)
    # Fetch the loss, accuracy, val_loss, and val_accuracy from the last epoch
    # (should be the last row in the CSV file unless there's something wrong
    # during the traning)
    lastEpochMetrics = epochMetrics.iloc[lastEpochNum, :].to_dict()
    # Attach the "batch_size" value
    lastEpochMetrics["batch_size"] = BS
    all_lastEpochMetrics.append(lastEpochMetrics)

In [ ]:
"""Now construct df_BS:""";

#df_BS = pd.DataFrame(all_lastEpochMetrics, #TODO)

#df_BS

### Step 4: Save the Intermediate DataFrame (BS)

In [ ]:
df_BS.to_csv("post_processing_hpc_bs.csv", index=False)

## Multiple Hidden Layers Experiment ("HL")

This experiment is different from the rest because now we alter the number of hidden neurons.
In the suggested hyperparameters to try (during the batch job submissions),
we vary only the number of hidden layers while fixing the number of neurons in each layer.
This equals increasing the depth of the network.

This experiment adds complexity in the naming convention.
We use the following model code string to denote the number of hidden layers and the num of neurons in each:

In [ ]:
def model_layer_code_XH(hidden_neurons):
    """Constructs a model-layer code string (e.g. 1H18N, 2H32N18N, ...).
    """
    hidden_neurons = list(hidden_neurons)
    hn_str = str(len(hidden_neurons)) + "H" \
           + "".join(str(HN) + "N" for HN in hidden_neurons)
    return hn_str

In [ ]:
# Try some

model_layer_code_XH([18])

In [ ]:
model_layer_code_XH([32,18])

### Step 1: Discover and Load the Results (HL)

In [ ]:
! ls -l -d scan-layers/model_*H*N_lr*_bs*_e*/

**EXERCISE** - Now create a list named `listHL` which contains the values of `hidden_neurons`'s used in your training runs:

In [ ]:
"""Enter the hidden layers configuration that you tried in your experiment,
see the contents of the `submit-scan-layers.sh` script in your working directory.""";

#dirPathHL = #TODO
#listHL = [ [18], [18,18], #TODO ]

### Step 2: Validation of Model Training: Visual Inspection (HL)

Let's repeat the visual validation of model training runs for the scan over hidden layers:

In [ ]:
"""Repeat the validation of model training, but now for the learning rate experiment""";

#for i, HL in enumerate(#TODO):
#    plot_training_history(fn_out_history_XH(#TODO),
#                          subtitle="hl"+model_layer_code_XH(HL))

**QUESTIONS**: Based on the plots shown above, inspect whether the training runs went as expected.

1) Visually inspect for any anomalies. In the answer box below, mark the runs that produce "abnormal training trends", i.e. where the "loss vs epochs" and/or "accuracy vs epochs" curves exhibit a different behavior from what shown in the first 2-panel plot near the top of this notebook.

2) Visually (or numerically) check for convergence (e.g. check the loss or accuracy for the last 4-5 epochs; what their slopes look like in this region; any fluctuations?)

3) Observe the differences in the *final* accuracies as a result of different `hidden_neurons` values. What happens with the metrics as we use deeper and deeper network? (We will do this more carefully in the next phase.)

> #### **ANSWERS**
>
> 1. #TODO
>
> 2. #TODO
>
> 3. #TODO

### Step 3: Create an Intermediate DataFrame for `hidden_layers` Scan

In [ ]:
"""Use the same loop structure we used earlier to parse the history data,
and create the temporary data structure which will be converted to
a dataframe called `df_HL` below.""";

#all_lastEpochMetrics = []
## Fill in the rows for the DataFrame
#for HL in listHL:
#    # Read the history CSV file and get the last row's data,
#    # which corresponds to the last epoch data.
#    #TODO
#    # Fetch the loss, accuracy, val_loss, and val_accuracy from the last epoch
#    # (should be the last row in the CSV file unless there's something wrong
#    # during the traning)
#    lastEpochMetrics = epochMetrics.iloc[lastEpochNum, :].to_dict()
#    # Attach the "hidden_neurons" value
#    lastEpochMetrics["hidden_neurons"] = str(list(HL))
#    all_lastEpochMetrics.append(lastEpochMetrics)

In [ ]:
"""Now construct df_HL:""";

#df_HL = pd.DataFrame(all_lastEpochMetrics, #TODO)

#df_HL

### Step 4: Save the Intermediate DataFrame (HL)

In [ ]:
df_HL.to_csv("post_processing_hpc_layers.csv", index=False)

## END of Post-processing Stage

Congratulations!
You have completed the post-processing of all the experiments you did previously.
In the next stage, we will perform analysis based on the post-processed result.
We will learn a lot of things regarding the roles of the key hyperparameters (hidden layers, learning rate, and batch size) in neural network models.